# Indexes and Ranges

* Getting more expressiveness around working with indexed data structures
* The new operators (^ and ..) are syntactic sugar. The functionality can be implemented by explicit calls to System.Index and System.Range factory methods
* Implicit Index support. Pattern based approach to get indexes and ranges out of the box for certain scenarios
* No special IL representation

```csharp
namespace System
{
    public readonly struct Index
    {
        public Index(int value, bool fromEnd);
    }
}
namespace System
{
    public readonly struct Range
    {
        public Range(System.Index start, System.Index end);
        public static Range StartAt(System.Index start);
        public static Range EndAt(System.Index end);
        public static Range All { get; }
    }
}

System.Index operator ^(int fromEnd);
System.Range operator ..(Index start = 0, Index end = ^0);
```


In [ ]:
(string name, string url)[] s_names =
{
    ("Archimedes", "http://totallyhistory.com/wp-content/uploads/2013/12/Archimedes.jpg"),
    ("Pythagoras", "http://totallyhistory.com/wp-content/uploads/2013/07/Pythagoras.jpg"), 
    ("Euclid", "https://www.thefamouspeople.com/profiles/images/euclid-2.jpg"),
    ("Socrates", "https://www.military-history.org/wp-content/uploads/2014/01/Socrates.jpg"),
    ("Plato", "http://janetsfox.com/wp-content/uploads/2015/12/Plato_Pio-Clemetino_Inv305.jpg")
};

Demo();

void Demo()
{
    // Provides a type- and memory-safe representation of a contiguous region of arbitrary memory.
    var names = s_names.AsSpan(); 
    display("0..^0 == .. == ..^0 == 0..");
    Display(names[..]);

    display("^2");
    Display(names[^2]);

    display("^2..^0");
    Display(names[^2..^0]);
}


void Display(ReadOnlySpan<(string name, string url)> source, bool asImage = true)
{
    foreach(var item in source)
    {
        Display(item, asImage);
    }
}
void Display((string name, string url) item, bool asImage = true)
{
    var _ = !asImage ? display(item) : display(img[src: item.url, style: "height: 100px;"]);
}


In [ ]:
var array = new int[] { 1, 2, 3, 4, 5 };

display(array);

display("[2..^3] == array[new Range(2, new Index(3, fromEnd: true))]");
var slice1 = array[2..^3];    // array[new Range(2, new Index(3, fromEnd: true))]
display(slice1);

display("[..^3] == array[Range.EndAt(new Index(3, fromEnd: true))]");
var slice2 = array[..^3];     // array[Range.EndAt(new Index(3, fromEnd: true))]
display(slice2);

display("[2..] == array[Range.StartAt(2)]");
var slice3 = array[2..];      // array[Range.StartAt(2)]
display(slice3);

display("[..] == array[Range.All]");
var slice4 = array[..];       // array[Range.All]
display(slice4);


// BANG 💥
display(array[^10]);

## Demo: running average

In [ ]:
public class StatInfo
{
    public string Label {get; set;}
    public int Min {get; set;}
    public int Max {get; set;}
    public double Average {get; set;}
}

int[] sequence = Sequence(1000);

IEnumerable<StatInfo> LeftToRightAverage()
{
    for(int start = 0; start < sequence.Length; start += 100)
    {
        Range r = start..(start+10);
        var (min, max, average) = MovingAverage(sequence, r);
        // Console.WriteLine($"From {r.Start} to {r.End}:    \tMin: {min},\tMax: {max},\tAverage: {average}");
        yield return new StatInfo() 
        {
            Label = $"From {r.Start} to {r.End}",
            Min = min, 
            Max = max,
            Average = average,
        };
    }
}

display(LeftToRightAverage().ToList());

IEnumerable<StatInfo> RightToLeftAverage()
{
    for (int start = 0; start < sequence.Length; start += 100)
    {
        Range r = ^(start + 10)..^start;
        var (min, max, average) = MovingAverage(sequence, r);
        // Console.WriteLine($"From {r.Start} to {r.End}:  \tMin: {min},\tMax: {max},\tAverage: {average}");
        yield return new StatInfo() 
        {
            Label = $"From {r.Start} to {r.End}",
            Min = min, 
            Max = max,
            Average = average,
        };
    }
}

display(RightToLeftAverage().ToList());

(int min, int max, double average) MovingAverage(int[] subSequence, Range range) =>
    (
        subSequence[range].Min(),b
        subSequence[range].Max(),
        subSequence[range].Average()
    );

int[] Sequence(int count) => Enumerable.Range(0, count).Select(x => (int)(Math.Sqrt(x) * 100)).ToArray();

## Demo

[sharplab-demo-ranges-indexes](https://sharplab.io/#v2:EYLgtghgzgLgpgJwDQBMQGoA+ABATARgFgAoACm3wAYACAOwjDiWopoFcEAbASgG0BdOgzhRqAXhIBvEtVnVSAIgCCCAMYALAJaMUIhcwXqYMAA4gA9OZgB7GBE6cAnltjWEjgHSrrYcwHcTAFpvWnhQ8zYTTmsIFChzXEp8AGZzfFxzFQ1tOF0oDwArEwBzBW4kGTlFAAVHGHUIYrdofWpDYzNLGzsHZ01Xdy8ffyCQsJgIqJi4hKTUygB2c1r6xub8otLy6krZRQBRNlVOTRRW9tMoC38/Pw96uAAzBms2KBM4ayi4Id8TBGsj00nBE5m0jVBcCOJxQgVwhRKZQqxDk8gUAGVrKoEBB4FBzkZLtdbncwMDNHZ3IEXDZBm5iiNgtZQnBwpForF4ol8AAWcxJcyY7G4kQIrbI1E1Ti46wEjrXAoQWhwGBQR7WAAev0ZY1ZE3Z0y5SQArGkMtVpTYAPrVTTWQIAYRBjBgmlo1itAElaAA3ZKUY1isokAC+AG4SCQAEpK4pwagIcTUDweAB6uAjxB9EET7yVSfojHySig6JMStI3EzVoLwigvAQ/GrSbztAbTZIADFrNYADzkKhCRjMVjUDg8AB8pELInbVa7Pf7o5nI8H4+4U9bvHhHn48+IkeI2B51G7fYAKlOo3BYgB5WhOMtK3uX6hQV5qODcKShoA)

```csharp
using System;
(string name, string url)[] names =
{
    ("Archimedes", "http://totallyhistory.com/wp-content/uploads/2013/12/Archimedes.jpg"),
    ("Pythagoras", "http://totallyhistory.com/wp-content/uploads/2013/07/Pythagoras.jpg"), 
    ("Euclid", "https://www.thefamouspeople.com/profiles/images/euclid-2.jpg"),
    ("Socrates", "https://www.military-history.org/wp-content/uploads/2014/01/Socrates.jpg"),
    ("Plato", "http://janetsfox.com/wp-content/uploads/2015/12/Plato_Pio-Clemetino_Inv305.jpg")
};

Range r = ..^2;
var span = names.AsSpan();
_ = names[r];
_ = span[r];
Foo<(string name, string url)>(names[r]);
Foo<(string name, string url)>(span[2..]);

void Foo<T>(ReadOnlySpan<T> source){}
```

# Reference

* <https://docs.microsoft.com/en-us/dotnet/csharp/tutorials/ranges-indexes>
* <https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/proposals/csharp-8.0/ranges>
